# Ch04. Unsupervised Learning

분류 회귀 문제에서 라벨링은 위압적인 작업일 수 있다.

라벨링이 불가능 한 상황도 있다.

비지도 학습은 패턴과 유사성 검출에 사용된다.

정규성과 비정규성의 패턴을 발견하는 것이 비지도 학습의 목표다

아래 알고리즘을 다룬다.

* K means: 관찰 피처 분류
* 기대 최대(EM): 관찰과 잠복 리퍼 분류
* 주 성분 분석(PCA): 모델의 차원 축소

K means는 scala에서 구현되었고 EM과 PCA는 Common math에서 구현됨

## Clustering

많은 수의 데이터와 피처들의 문제는 갈수록 다루기 어렵게 되고 있다. 

많은 엔지니어링 분야가 큰 데어터 집합을 분류하는데 분할 정복을 사용하고 있다. 연속적이고 무한의 거대 데이터집합을 줄이는것이 목표가 되고 있다.

<img src='./figures/VisualizationOfDataClustering.png'>

이 기법은 벡터 양자화라고 불리고 관측 데이터를 작은 크기의 그룹으로 나누는 방법이다.이 방법의 혜택은 각 그룹의 대표성을 사용한 분석은 전체 데이터집합을 분석하는 것보다 간단하다는 것이다.

벡터 양자화는 클러스터라는 그룹을 생성하기 위해 거리와 유사성 개념에 의존한다.

### Learning vector quantization (LVQ)

Vector quantization는 vector quantization 학습과는 다르다. vector quantization은 승자독식 학습에 의존하는 인공 신경만의 특별한 경우이다.

이 장은 대표적인 두개의 알고리즘을 소개한다.

* K-means, which is used for quantitative types and minimizes the total error (known as the reconstruction error) given the number of clusters and the distance formula.
* Expectation-maximization (EM), which is a two-step probabilistic approach that maximizes the likelihood estimates of a set of parameters. EM is particularly suitable to handle missing data.

### K-means clustering

K-means는 각 클러스터의 대표가 클로이드라고 불리는 클러스터의 중심으로 계산되는 대중적인 반복적 군집 알고리즘이다. 군집내에 거리를 기준으로 계산한다.

### Measuring similarity

관측치 간에 유사성을 측정하는 방법은 많으나 가장 적절한 측정은 직관적이어야 하고 계산 복잡도를 피해야 한다. 

세가지 유사성 측정을 고려해 보면

1. The Manhattan distance
2. The Euclidean distance
3. Cosine of value observations

The Manhattan distance는 변수간의 절대 거리로 계산 {x i } and {y i }

<img src='./figures/manhattan.png'>

In [5]:
def manhattan[T <% Double, U <% Double]
    (x: Array[T], y: Array[U]): Double = 
        (x,y).zipped.foldLeft(0.0)((s, t) => s + Math.abs(t._1 - t._2))

defined function manhattan

In [2]:
List(1,2,3).zip(List(3,4,5))

res0: List[(Int, Int)] = List(
  (1, 3),
  (2, 4),
  (3, 5)
)

The ubiquitous Euclidean distance는 두 벡터의 거리의 제곱의 제곱근으로 구한다.
{x i } and {y i }, of the same size:

<img src='./figures/euclidean.png'>

In [6]:
def euclidean[T <% Double, U <% Double]
    (x: Array[T], y: Array[U]): Double =
        Math.sqrt((x, y).zipped.foldLeft(0.0)((s, t) => { val d = t._1 - t._2; s + d*d} ))


defined function euclidean

The cosine distance는 두 벡터의 코사인 각도로 정의된다. {x i } and {y i },
of the same size:

<img src='./figures/cosine.png'>

In [ ]:
def cosine[T <% Double, U <% Double](xs: Array[T], ys: Array[T]): Double = {
    val zeros = (0.0, 0.0, 0.0)
    val sss = (xs, ys).zipped.foldLeft(zeros)(
        {
        (s, t) => 
            (s._1 + t._2 * t.3, s._2 + t._2 * t._2, s._3 + t._3* t._3)
        }
    )
    sss._1 / Math.sqrt(sss._2 * sss._3)
}

### Tip

scalar의 dot product는 엄청 일반적인 경우이고 scala에서는 zip을 통해서 dot product를 구현한다.

In [3]:
def dot (x:Array[Double], y:Array[Double]): 
    Array[Double] = x.zip(y).map( x => f(x._1, x._2) )

: 

In [4]:
def dot(x:Array[Double], y:Array[Double]): 
    Array[Double] = (x, y).zipped map ( _ * _)

defined function dot

### Overview of the K-means algorithm

the K-means 알고리즘의 큰 이점은 유사성이다.

#### Note

K clusters {C k } 

means {m k }. 

The K-means 알고리즘은 최적화 문제라서 목적 자체가 재구조를 최소화하고 전체 오류를 줄이는 것이다.

<img src='./figures/clustering.png'>

반복 알고리즘의단계는 

* Initialize the centroids or means m k of the K clusters.
* Assign observations to the nearest cluster given m k .
* Iterate until no observations are reassigned to a cluster:
  * Compute centroids m k that minimize the total error reconstruction for the current assignment
  * Reassign the observations given the new centroids mk

### Step 1 – cluster configuration

K개의 군집 수 정하기와 군집내 센트로이드 초기화 작업이 필요

#### Defining clusters

The first step is to define a cluster. A cluster is defined by the following parameters:
* Centroid: center
* The indices of the observations that belong to this cluster: members

In [12]:
import scala.collection.mutable._

type DblVector = Array[Double]

class Cluster[T <% Double](val center: DblVector) {
    val members = new ListBuffer[Int]
}

import scala.collection.mutable._
defined type DblVector
defined class Cluster

클러스터는 이터레이션 계산에서 멤버를 관리한다. 한번에 두개 클러스터에 존재하지 못한다.

In [13]:
object Cluster {
    def apply[T <% Double](c:DblVector):Cluster[T] = new Cluster[T](c)
}

defined object Cluster

클러스터는 관측치의 멤버를 관리하거나 갱신할 수 있고 관측 멤버의 분산과 표준 편차를 구할 수 있다.

In [4]:
type XTSeries[T] = Array[T]

def += (n:Int): Unit = members.append(n)
def moveCenter(xt: XTSeries[Array[T]): Cluster[T] ={
    val sums = members.map(xt(_).map(_.toDouble)).
        toList.transpose.map( _.sum)    
    Cluster[T](sums.map( _ / members.size).toArray)
}
def stdDev(xt: XTSeries[Array[T]], 
           distance: (DblVector, Array[T]) => Double): Double =
{
    Stats[Double](members.map(xt( _)).
        map( distance(center, _)).toArray).stdDev
}

: 

세가지 중요한 함수가 있다.

* += : 클러스터에 멤버 추가
* moveCenter : 기존 멤버들에서 새로운 센트로이드를 계산하여 새로운 클러스터를 준다.
* stdDev : 표준 편차나 밀집도를 계산해 준다.

#### Note

##### Cluster selection

관측치를 재배정할 때 클러스터를 선택하는 방법들이 있다. 표준 편차가 크고 밀도가 낮은 넘을 선택하게 되는데 우리는 멤버가 많은 크러스터를 대안으로 선택할 것이다.

##### Defining K-means

In [13]:
class KMeans[T <% Double](K: Int, maxIters: Int, 
        distance: (DblVector,Array[T]) => Double)
        (implicit order: Ordering[T], m: Manifest[T]) extends
            PipeOperator[XTSeries[Array[T]], List[Cluster[T]]] {
    def |> : PartialFunction[XTSeries[Array[T]], List[Cluster[T]]]
    def initialize(xt:XTSeries[Array[T]]): List[Cluster[T]]

: 

다른 데이터 처리처럼 K means도 파이프 |>를 사용한다. 2장에서 배운 종소성 주입이 클러스터에 통합되었다. 초기화 함수로 센트로이드가 설정된다.

#### Initializing clusters

센트로이드의 초기화는 K means의 빠른 수렴을 위해서 중요하다. 센트로이드 후보를 적합을 평가하는 유전자 알고리즘 군에 속한다.

초기화 과정:

1. Compute the standard deviation of the set of observations.
2. Select the dimension k {x k , 0 , x k , 1 ... x k , n } with maximum standard deviation.
3. Rank the observations by their increasing value of standard deviation for the dimension k.
4. Divide the ranked observations set equally into K sets {S m }.
5. Find the median values, size (S m )/2.
6. Use the corresponding observations as centroids.

In [14]:
def initialize(xt:XTSeries[Array[T]]): List[Cluster[T]]={
    val stats = statistics(xt) //1
    val maxSDevDim = Range(0,stats.size).maxBy (stats( _ ).stdDev)//2
    val rankedObs = xt.zipWithIndex
    .map(x=> (x._1(maxSDevDim), x._2)) //2
    .sortWith( _._1 < _._1) //3
    val halfSegSize = ((rankedObs.size>>1)/K).floor.toInt //4
    val centroids = rankedObs.filter(isContained( _, halfSegSize,
    rankedObs.size) ).map(n => xt(n._2)) //6
    Range(0, K).foldLeft(List[Cluster[T]]())((xs, i) => Cluster[T](centroids(i)) :: xs) //7
}

: 

초기화에는 Agha-Ashour 알고리즘이 쓰였다.

http://www.mecs-press.org/ijisa/ijisa-v4-n1/IJISA-V4-N1-3.pdf

In [14]:
def isContained(t: (T,Int), hSz: Int, dim: Int): Boolean =
    ((t._2 % hSz == 0) && (t._2 %(hSz<<1) != 0)

: 

#### Step 2 – cluster assignment

두번째 단계는 클러스터에 관측치를 배정한다.

In [16]:
def assignToClusters(xt: XTSeries[Array[T]], 
                     clusters: List[Cluster[T]], 
                     membership: Array[Int]): Int = {
    xt.toArray
        .zipWithIndex
        .filter(x => { //1
        val nearestCluster = getNearestCluster(clusters, x._1)//2
        val reassigned = nearestCluster != membership(x._2)
        clusters(nearestCluster) += x._2 //3
        membership(x._2) = nearestCluster //4
        reassigned
    }).size
}

: 

배정의 핵심은 가장 가까운 클러스터의 인덱스를 계산하는 필터에 있다.

최근거리를 계산하는 메소드:

In [16]:
def getNearestCluster(clusters: List[Cluster[T]], x:Array[T]): Int={
    clusters.zipWithIndex.foldLeft((Double.MaxValue,0))((p,c) => {
    val measure = distance(c._1.center, x)
    if(measure < p._1) (measure, c._2) else p
    })._2

: 

#### Step 3 – iterative reconstruction

마지막 단계는 재구축 오류를 계산하는 반복자를 구현하는 것이다.

K means 클러스터 알고리즘에서는 파이프 |>에 반복자 추출이 포함되어 있다.

In [16]:
def |> :PartialFunction[XTSeries[Array[T]], List[Cluster[T]]] = {
    case xt: XTSeries[Array[T]] if(xt.size>2 && xt(0).size>0) => {
        val clusters = initialize(xt) //1
        if( clusters.isEmpty) List.empty
        else {
            val membership = Array.fill(xt.size)(0)
            val reassigned = assignToClusters(xt,clusters,membership)//2
            var newClusters: List[Cluster[T]] = List.empty
            Range(0, maxIters).find( _ => {
                newClusters = clusters.map( c => {
                    if( c.size > 0) c.moveCenter(xt, dimension(xt))
                    else clusters.filter( _.size > 0)
                        .maxBy( _.stdDev(xt, distance))
                    }) //3
                assignToClusters(xt, newClusters, membership) == 0
            }) match {
            case Some(index) => newClusters
            case None => { ... }
        } //4
    }
}

: 

#### Tip

##### K-means algorithm exit condition

드물지만 알고리즘은 동일한 관측치를 재배정하면서 가끔 수렵하지 못하게 막을 수 도 있다.

그래서 이터레이션의 최대치를 정하여 탈출 조건을 추가하기 권한다. 만일 K-means가 최대 반복에도 수렴하지 못하면 첨부터 다시해야 한다.

In [17]:
def apply[T <% Double](K: Int, maxIters: Int)
    (implicit order: Ordering[T], m: Manifest[T]): KMeans[T] = 
    new KMeans[T](K, maxIters, euclidean)

def stdDev[T](c: List[Cluster[T]], xt: XTSeries[Array[T]]): 
    List[Double] = c.map( _.stdDev(xt))

: 

#### Note

##### Centroid versus mean

센트로이드와 평균은 값은 개체를 참조한다. 이제부터는 둘을 혼용해서 사용할 거다.

ordering a trait와 Manifest는 apply 생성자에서 제공된다. 왜서냐면? 클라이언트 코드에서 런타임에 제공 받는단 보장이 없기에

### Curse of dimensionality

의미 있는 피처의 수를 위해서는 모델은 거대 관측치를 필요로 한다.

50보다 작은 매우 작은 수의 관측치로 구성된 K means 클러스터링은 상당히 편의된 모델을 만들어 낸다.

저자는 경험상 다음의 규칙을 사용한단다.
훈련 집합:n, 기대 클러스터: K, 피처: N

n < K.N.

#### Note

##### Dimensionality versus size of training set

모델의 차원대 관측치의 수에 따른 이슈는 비지도학습 알고리즘에서는 특별하지 않다. 지도 학습에서는 viable training plan을 구축해서 동일한 과제를 대한다.


<img src='./figures/PriceModelForKMeansClustering.png'>

제한을 두거나 관측치를 줄인다.

* Sampling the trading data without losing a significant amount of information from the raw data, assuming the distribution of observations follows a known probability density function.
* Smoothing the data to remove the noise as seen in Chapter 3, Data Preprocessing, assuming the noise is Gaussian. In our test, a smoothing technique will remove the price outliers for each stock and therefore reduce the number of features (trading session). This approach differs from the sampling approach because it does not require an assumption that the dataset follows a known density function. On the other hand, the reduction of features will be less significant.

이런 접근법이 이 책의 목적상 최고의 솔루션이라 할 수 있다. 하지만 실제 상업용 분석에서는 권하지 않는다. 이후에 소개될 PCA가 신뢰할 수 있는 차원 축소 기법중에 하나이다.

#### Experiment

13년 1월에서 12월까지 일별 주식 종가의 클러스터를 뽑는 것이 목적이다. 127개 종목은 S&P 500에서 무작위로 추출됐고 아래 시각화했다.

<img src='./figures/PriceActionOfStocksUsedInKMeansClustering.png'>

핵심은 클러스터링에 앞서 적절한 피처를 정하는 것이다. 전체 가격 이력을 고려하여 252일치를 피처로 선정했다. 하지만 관측치 수는 전체 가격 범위를 확인하기 위해서는 너무 제한적이다. 50개의 관측치는 80일에서 130일 사이의 주식 종가이다. 일별 종가는 민,맥스를 사용해서 정규화했다.

먼저, K means 알고리즘을 실행하는 간단한 함수를 정의하자.

In [18]:
Val MAX_ITERS = 150
def run(K: Int, obs: DblMatrix): Unit = {
    val kmeans = KMeans[Double](K, MAX_ITERS) //1
    val clusters = kmeans |> XTSeries[DblVector](obs) //2
    clusters.foreach( _.center.foreach( show( _ ))) //3
    clusters.map( _.stdDev(XTSeries[DblVector](obs, euclidean))).foreach( show( _ ) )
    //4
}

: 

The KMeans class is first initialized with a number of clusters, K, and a maximum number of iterations, MAX_ITERS (line 1). These two parameters are domain and problem specific.
The clustering algorithm is executed (line 2) returning a list of clusters. The clusters'centroid information is then displayed (line 3) and the standard deviation is computed for each of the clusters for a given number of clusters, K, and observations, obs (line 4).

Let's load the data from CSV files using the DataSource class (refer to the Data extraction section in Appendix A, Basic Concepts):

In [18]:
final val path = "resources/data/chap4/"
val extractor = YahooFinancials.adjClose :: List[Array[String] =>Double]() // 5
def symbols = DataSource.listSymbols(path) //6
final val START = 80
final val SAMPLES = 50
val normalize=true
val prices = symbols.map(s =>DataSource(s,path,normalize) |> extractor) //7
prices.find(_.isEmpty) match {
    //8
    case Some(noPrice) = { ... }
    case None => {
        val values = prices. map(x => x(0))
        .map(_.drop(START).take(SAMPLES))
        args.map(_.toInt) foreach( run(_, values)) //9
    }
}

: 

K=3일 때 군집 분석이다. 각 클러스터의 평균 벡터는 아래 그림과 같다.

<img src='./figures/ChartOfMeansOfClustersUsingKMeansK3.png'>

세개의 클러스터의 평균벡터는 확연히 구분된다. 맨위와 맨 아래는 1, 2번이다. sd가 0.34와 0.27로 매우 유사한 패턴을 보인다. 1과 2의 요소간에 차이는 거의 0.37이다. 클러스터 3은 시작시점에는 2번과 유사하게 나오지만 뒤로 갈 수록 3번에 가까운 패턴을 보인다.

시간 윈도와 거래 기간 (80~130)에서 동작을 쉽게 설명해 준다. 연방 준비의 정책을 그대로 반영하고 있다.

<img src='./figures/ListOfStocks.png'>

클러스터 수에 따른 영향도를 평가해 보자.

#### Tuning the number of clusters

2에서 15까지 클러스터 수를 조절할 것이다.

2부터 시작해 보자.

<img src='./figures/ChartOfMeansOfClustersUsingKMeansK2.png'>

라벨 2는 3개 클러스터의 라벨 3와 비슷하다. 하지만 라벨 1은 3개 클러스터의 라벨 1, 2를 통합한 것과 비슷하다.

통합 효과는 왜 클러스터 1의 표준 편차인 0.55가 클러스터 2의 표준편차 0.28에 두배가 되는지 보여 준다.

이번에 5개 클러스터이다.

<img src='./figures/ChartsOfMeansOfClustersUsingKMeansK5.png'>

이번 차트에서 클러스터 1, 2, 3는 3개 클러스터의 동일한 라벨의 클러스터와 비슷하다. 벡터 4인 클러스터는 클러스터 3과 유사하지만 방향이 뒤집혔다. 다시 말해 3, 4 클러서트는 금융정책에 반대로 반응한 사례이다.

10개 클러스터 짜리이다.

<img src='./figures/ChartOfMeansOfClustersUsingKMeansK10.png'>

k=3일 때 나온 본 클러스터 1,2,3의 평균은 여전히 나오고 있다. 가장 신뢰할 수 있는 클러스터로 가정할 수 있다. 이 클러스터는 낮은 표준 편차와 높은 밀도를 가진다.

centroid c j인 cluster C j의 밀도를 유클리디안 거리의 역수로 정의할 수 있다.

<img src='./figures/DistanceOfEuclidean.png'>

이번엔 13개 이다.

<img src='./figures/BarChartOfTheAverageClusterDensityForK1to13.png'>

기대했던바와 같이 평균 밀도는 K가 증가하면 따라서 증가한다. K=5일 때까지는 의미 있게 증가하다가 이후에는 항상 증가하지는 않는다.

비정상적인 3개 요일을 설명하자면:
* The original data is noisy
* The model is somewhat dependent on the initialization of the centroids
* The exit condition is too loose

#### Validation

##### Tip

##### Validation

클러스터의 품질은 F1으로 측정된다. F1이 높은 모델을 찾아 정한다.

결과 품질에 영향을 주는 조절 파라미터를 되돌아 보면:

* Initial selection of centroid
* Number of K clusters

일부에선 유사성 기준은 명료성과 클러스터 밀도에 영향을  줄 수 있다.

최종적으로 중요한 고려사항은 K means의 계산 복잡도이다.

이런 장점에더 불과하고 결측치나 미관측치 정보는 다룰수 없고 피처는 서로에게 영향을 미친다.

### Expectation-maximization (EM) algorithm

EM은 maximum likelihood를 추정하기 위해 소개돼었다. 관측되지 않은 값이 있는 우도를 최대화 하는 모델의 피처를 계산하기 위함이다.

반복 알고리즘은 다음으로 구성된다:

* The expectation, E, of the maximum likelihood for the observed data by inferring the latent values (E-step)
* The model features that maximize the expectation E (M-step)

EM 알고리즘은 표중 가우시안 분포를 따르는 클러스터 문제를 풀때 적용된다.

### Gaussian mixture model

잠복 변수 Z는 행위의 근본인 Z의 모델 X의 행위로 가시되할 수 있다.

<img src='./figures/VisualizationOfObservedAndLatentFeatures.png'>

#### Note

##### The mixture model

{xi}가 잠복 피처 {zi}와 결함된 관찰 피처이면 z가 j로 주어졌을 때 피처 xi를 위한 확률은?

<img src='./figures/pxi.png'>

확률 p는 기저 분포라 한다. 전체 모델 θ= {x i , z k }로 확장하면 조건부 확률은 

<img src='./figures/pxisigma.png'>

가장 넓게 사용되는 믹스처 모델은 기저분포 P를 정규 분포로, 조건 확률을 가중 정규 다변량 분포로 표현할 수 있는 가우시안 믹스처 모델이다.

<img src='./figures/pxisigma-1.png'>

#### EM overview

구현에 관한한 EM 아고리즘은 3단계를 거친다.

* The computation of the log likelihood for the model features given some latent variables (LL).
* The computation of the expectation of the log likelihood at iteration t (E-step).
* The maximization of the expectation at iteration t (M-step).

#### Note

#### Log likelihood

* LL: 관차 변수 X = {xi}와 잠복 변수 Z={zi}를 고려하자. 주어진 z에 대한 x의 로그 라이클리후드는

<img src='./figures/ll.png'>

* E-step: 모델 변수 θ에서의 기대값은 다음으로 계산된다.

<img src='./figures/estep.png'>

* M-step: 함수 Q는 모델 피처 θ를 위해서 최대화된다. Borman's tutorial.

<img src='./figures/mstep.png'>

#### Implementation

##### Tip

##### Inner workings of EM

아파치 컴먼스에서 계산한다. 관심있음 보시길.

In [ ]:
type EM = MultivariateNormalMixtureExpectationMaximization
type EMOutput = List[(Double, DblVector, DblVector)]
import scala.collections.JavaConversions._ //1

The constructor of the MultivariateEM class uses the standard template for machine learning algorithm classes:

* Parameterized view bound type
* Implementation of EM as a data transformation by extending PipeOperator

Here is an implementation of the constructor of MultivariateEM:

In [ ]:
class MultivariateEM[T <% Double](K: Int) extends
PipeOperator[XTSeries[Array[T]], EMOutput]

자바 컬랙션을 스칼라 컬랙션으로 변환시켜 줘야 한다. (line 1).

EM 알고리즘 구현은 |> 오퍼레이터가 필요하다.

In [19]:
def |> : PartialFunction[XTSeries[Array[T]], EMOutput] = {
    case xt: XTSeries[Array[T]] if(xt.size>0 && dimension(xt)>0) =>{
        val data: DblMatrix = xt
        //2
        val multivariateEM = new EM(data)
        val est = MultivariateNormalMixtureExpectationMaximization
            .estimate(data, K)
        multivariateEM.fit(est) //3

        val newMixture = multivariateEM.getFittedModel //4
        val components = newMixture.getComponents.toList //5
        components.map(p => (p.getKey.toDouble, p.getValue.getMeans,
        p.getValue.getStandardDeviations)
    ))//6
    ....

: 

#### Tip

#### Third-party library exceptions

스칼라는 exception을 method 선언에 넣지 않는다. (java와 다름)
로컬에서 예외가 처리된다는 보장이 없음. 이문제를 예외가 두개의 시나리오의 3rd 라이브러리에서 발생한다.

* The documentation of the API does not list all the types of exceptions
* The library is updated and a new type of exception is added to a method

가장 간단한 스칼라의 예외 처리 메커니즘은 :

In [19]:
Try {
    ..
} match {
    case Success(results) => ...
    case Failure(exception) => ...
}

: 

#### Testing

주식에 EM을 적용해 보자.

EM 알고리즘으로 분석할 주식의 수는 사용된 관측수를 제한한다. 간단한 옵션은 주식의 노이즈 중 일부를 거른다.


#### Tip

#### Filtering and sampling

클러스터링 전에 Moving average와 고정 간격 표본법을 조합한 전처리는 매우 기초적인 것들이다. 예를 들어 과거의 주식 이력은 동일한 노이즈 성격을 보인다.



In [ ]:
val extractor = YahooFinancials.adjClose :: List[Array[String] =>Double]() //1
val period = 8
val samplingRate = 10
val smAv = SimpleMovingAverage[Double](period) //2
val obs = DataSource.listSymbols(path).map(sym => { //3
    val xs = DataSource(sym, path, true) |> extractor //2
    val values : XTSeries[Double] = XTSeries.|>(xs)).head //4
    val filtered = smAv |> values
    filtered.zipWithIndex //5
        .drop(period+1).toArray //6
        .filter( _._2%samplingRate==0)
        .map( _._1)
})

<img src='./figures/ChartOfTheNormalizedMeansPerClusterUsingEMK3.png'>

클러스터 2, 3은 유사한 패턴을 가진다. 두개는 동일하거나 유사한 통찰을 보인다. 다음은 표준 정규 분포화한 그림이다.

<img src='./figures/ChartOfTheNormalizedStandardDeviationPerClusterUsingEMK3.png'>

각 클러스터의 평균 벡터의 표준 편차의 분포는 다른 하나의 분포가 연방 준비은행의 공지에 따라 상승할 때 두개 산업의 주가가 동반 하락한다는 사실을 설명해 준다. 

#### Note

##### Relation to K-means

You may wonder what is the relation between EM and K-means as both techniques address
the same problem. The K-means algorithm assigns each observation uniquely to one and
only one cluster. The EM algorithm assigns an observation based on posterior probability.
K-means is a special case of the EM for Gaussian mixtures [4:11].

#### Online EM

Online learning is a powerful strategy for training a clustering model when dealing with
very large datasets. This strategy has regained interest from scientists lately. The
description of online EM is beyond the scope of this tutorial. However, you may need to
know that there are several algorithms available for online EM if you ever have to deal with
large datasets: batch EM, stepwise EM, incremental EM, and Monte Carlo EM
[4:12].

### Dimension reduction

정보 영역의 사전 지식없이 데이터 과학자는 가능한 모든 피처를 포함하려 한다. 그것도 첫 분석에서, 결국에는 가정을 만들고 이는 열악하고 위험한 차원 축소를  하게된다.

모델이 수백개의 피처를 사용하는게 비일반적이진 않다. 노이즈 필터링 기법은 피처에 대한 모델의 민감도를 떨어뜨린다. 하지만 이런 노이즈에 연관된 피처는 훈련 과정 전에는 알려저 있지 않아 버려질 수 없다. 결과적으로 모델 훈련은 매우 부담스럽게 된다.

과적합은 거대한 피처 집합에는 또 다른 장애이다. 제한된 크기의 훈련 집합은 거대한 피처로 모델을 만드는 것을 허락하지 않는다.

차원 축소 기술은 이런 문제를 해결한다.

여기 차원 축소에 3가지 접근법이 있다.

* Statistical analysis solutions such as ANOVA for smaller feature sets
* Regularization and shrinking techniques, which are introduced in Chapter 6, Regression and Regularization
* Algorithms that maximize the variance of the dataset by transforming the covariance matrix


### Principal components analysis (PCA)

차원 축소의 목적은 변수의 차원을 줄여서  원본  피처를 새롭게 정렬된 피처로 변환하는 것이다. 원본 관측은 더 낮은 차원으로 변환된다.

Let's consider a model with two features {x, y} and a set of observations {xi, yi} plotted on
the following chart:

<img src='./figures/VisualizationOfPrincipalComponentsForA2DimensionModel.png'>

x,y는 더 적절히 매칭되는 X,Y로 변환된다. 설명력이 높은 변수를 first principal component라 불린다. n번쨰를 n principal component라 부른다.

#### Algorithm

Lindsay Smith 의 강좌를 추천한다.

#### Note

PCA와 공분산 메트릭스는 두개의 피처 X, Y의 공분삭이다.

<img src='./figures/pca1.png'>

여기서 x,y에 대한 각각의 평균이다.

공분산은 zScore로 계산된다.

<img src='./figures/pca2.png'>

n개 피처로 만든어 진 모델에서 공분산 메트릭스는 아래와 같다.

<img src='./figures/pca3.png'>

x에서  X로 변환은 공분산 메트릭스의 아이젠벨류로 계산된다.

<img src='./figures/pca4.png'>

아이젠 벨류는 분산과 누적 분삭의 감소 차수로 rank된다. 마지막으로 m의 최상위 아이젠 벨류는 기 정의된 문턱값을 넘는 값이다.

<img src='./figures/pca5.png'>

알고리즘은 5단계로 구현된다.

* Compute the zScore for the observations by standardizing the mean and standard deviation.
* Compute the covariance matrix Σ for the original set of observations.
* Compute the new covariance matrix Σ' for the observations with the transformed features by extracting the eigenvalues and eigenvectors.
* Convert the matrix to rank eigenvalues by decreasing the order of variance. The ordered eigenvalues are the principal components.
* Select the principal components for which the total sum of variance exceeds a threshold by as a percentage of the trace of the new covariance matrix.

공분사 메트릭스의 대각화에 의한 PC의 추출은 아래에 가시화된다. 공분산 수치를 위해 컬러가 들어 갔다.

<img src='./figures/VisualizationOfTheExtractionOfEigenvaluesInPCA.png'>

아이젠벨류는 차원을 축소로 rank된다. PCA알고리즘은 누적 값이 무의미할때 종료된다.

PCA는 Apache Commons Math lib에 있다.

In [ ]:
import types.ScalaMl._, types.CommonMath._, //2

def |> : PartialFunction[XTSeries[Array[T]], (DblMatrix, DblVector)]={
    case xt: XTSeries[Array[T]] if(xt !=null && xt.size>1) => {
        zScoring(xt) match {//1
    case Some(obs) => {
        val covariance = new Covariance(obs).getCovarianceMatrix //3
        val transf = new EigenDecomposition(covariance)
        val eigVectors = transf.getV //4
        val eigValues = new ArrayRealVector(transf.getRealEigenvalues)
        val cov = obs.multiply(eigVectors).getData
        (cov, eigValues.toArray) //5
        ...

PCA는 원 집합을 표준화해서 사용한다. 공분산을 구하고, 아이젠분석을 해서 벡터의 값을 반환한다.

#### Test case

PCA알고리즘을 34개 재무 척도로 표현하면

1. Trailing Price-to-Earnings ratio (PE)
2. Price-to-Sale ratio (PS)
3. Price-to-Book ratio (PB)
4. Return on Equity (ROE)
5. Operation Margin (OM)


val data = Array[(String, DblVector)] (
// Ticker
PE
PS
PB
ROE
OM
("QCOM", Array[Double](20.8, 5.32, 3.65, 17.65,29.2)),
("IBM",
Array[Double](13, 1.22, 12.2, 88.1,19.9)),
...
)


PCA를 실행핳는 클라이언트 코드:

val pca = new PCA[Double] //1
val input = data.map( _._2.take(3))
val cov = pca |> XTSeries[DblVector](input) //2
Display.show(toString(cov), logger) //3

pca생성, 데이터에서 pca 변환 수행하여 공분산 추출, 화면에 출력

### Evaluation

34개 재무 비율은 5차원 모델을 사용한다. 그 고유값은

2.5321, 1.0350, 0.7438, 0.5218, 0.3284

고유값에 상대적인 값은 아래 그림에 있다.

<img src='./figures/DistributionOfEigenvaluesInPCAFor5Dimensions.png'>

The chart shows that 3 out of 5 features account for 85 percent of total variance (trace of
the transformed covariance matrix). I invite you to experiment with different combinations
of these features. The selection of a subset of the existing features is as simple as applying
Scala's take or drop methods:

Val numFeatures = 4
val ts = XTSeries[DblVector](data.map(_._2.take(numFeatures)))

Let's plot the cumulative eigenvalues for the three different model configurations:
* Five features: PE, PS, PB, ROE, and OM
* Four features: PE, PS, PB, and ROE
* Three features: PE, PS, and PB

<img src='./figures/DistributionOfEigenvaluesInPCAFor345Features.png'>

The chart displays the cumulative value of eigenvalues that are the variance of the
transformed features Xi. If we apply a threshold of 90 percent to the cumulative variance,
then the number of principal components for each test model is as follows:
* {PE, PS, PB}: 2
* {PE, PS, PB, ROE}:3
* {PE, PS, PB, ROE, OM}: 3

In conclusion, the PCA algorithm reduced the dimension of the model by 33 percent for
the 3-feature model, 25 percent for the 4-feature model, and 40 percent for the 5-feature
model for a threshold of 90 percent.

#### Note

##### Cross-validation of PCA

Like any other unsupervised learning technique, the resulting principal components have
to be validated through a one or K-fold cross-validation using a regression estimator such
as partial least square regression (PLSR) or the predicted residual error sum of
squares (PRESS). For those not afraid of statistics, I recommend Fast Cross-validation
in Robust PCA by S. Engelen and M. Hubert [4:14]. You need to be aware, however, that
the implementation of these regression estimators is not simple.

The principal components can be validated through a 1-fold or K-fold cross-validation, by
performing some type of regression estimators or EM on the same dataset. The validation
of the PCA is beyond the scope and space allocated to this chapter.

Principal components analysis is a special case of the more general factor analysis. The
later class of algorithm does not require the transformation of the covariance matrix to be
orthogonal.

#### Other dimension reduction techniques

Although quite popular, the principal components analysis is far from being the only
dimension reduction method. Here are some alternative techniques, listed as reference:
factor analysis, principal factor analysis, maximum likelihood factor analysis, independent
component analysis (ICA), Random projection, nonlinear PCA, nonlinear ICA, Kohonen's
self-organizing maps, neural networks, and multidimensional scaling, just to name a few
[4:15].

## Performance considerations

The three unsupervised learning techniques share the same limitation—a high
computational complexity.

### K-means

The K-means has the computational complexity of O(iKnm), where i is the number of
iterations, K the number of clusters, n the number of observations, and m the number of
features. The algorithm can be improved through the use of other techniques by using the
following techniques:
* Reducing the average number of iterations by seeding the centroid using an
algorithm such as initialization by ranking the variance of the initial cluster as
described at the beginning of this chapter.
* Using a parallel implementation of K-means and leveraging a large-scale framework
such as Hadoop or Spark.
* Reducing the number of outliers and possible features by filtering out the noise with
a smoothing algorithm such as a discrete Fourier transform or a Kalman filter.
* Decreasing the dimensions of the model by following a two-step process: a first pass
with a smaller number of clusters K and/or a loose exit condition regarding the
reassignment of data points. The data points close to each centroid are aggregated
into a single observation. A second pass is then run on a smaller set of observations.

### EM

The computational complexity of the expectation-maximization algorithm for each
iteration (E + M steps) is O(m 2 n), where m is the number of hidden or latent variables and
n is the number of observations.

A partial list of suggested performance improvement includes:
* Filtering of raw data to remove noise and outliers
* Using a sparse matrix on a large feature set to reduce the complexity of the
covariance matrix, if possible
* Applying the Gaussian mixture model (GMM) wherever possible: the
assumption of Gaussian distribution simplifies the computation of the log likelihood
* Using a parallel data processing framework such as Apache Hadoop or Spark as
explained in the Apache Spark section in Chapter 12, Scalable Frameworks
* Using a kernel method to reduce the estimate of covariance in the E-step


### PCA

The computational complexity of the extraction of the principal components is O(m 2 n +
n3), where m is the number of features and n the number of observations. The first term
represents the computational complexity for computing the covariance matrix. The last
term reflects the computational complexity of the eigenvalue decomposition.

The list of potential performance improvements or alternative solutions for PCA includes:
* Assuming that the variance is Gaussian
* Using a sparse matrix to compute eigenvalues for problems with large feature sets
and missing data
* Investigating alternatives to PCA to reduce the dimension of a model such as the
discrete Fourier transform (DFT) or singular value decomposition (SVD)
[4:16]
* Using the PCA in conjunction with EM (a research)
* Deploying a dataset on a parallel data processing framework such as Apache Spark
or Hadoop as explained in the Apache Spark section in Chapter 12, Scalable
Frameworks

### Summary

This completes the overview of three of the most commonly used unsupervised learning
techniques:
* K-means for clustering fully observed features of a model with reasonable
dimensions
* Expectation-maximization for clustering a combination of observed and latent
features
* Principal components analysis to transform and extract the most critical features in
terms of variance

The key point to remember is that unsupervised learning techniques are used:
* By themselves to extract structures and associations from unlabelled observations
* As a preprocessing stage to supervised learning in reducing the number of features
prior to the training phase

In the next chapter, we will address the second use case, and cover supervised learning
techniques starting with generative models.

# Thank you so much, Q & A